This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "voc-1831442288126677345071966cb5bc64e3733.92039541"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

In [2]:
model_name = "gpt-3.5-turbo"
temp = 0.0
model = OpenAI(model_name = model_name, temperature = temp, max_tokens = 2000)

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [3]:
model

OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_kwargs={'temperature': 0.0, 'max_tokens': 2000})

In [4]:
template = """
Your Role is a real estate expert
Generate {num_listings} real estate listing and make sure to include
in each listing : Neighbourhood Name, Price, Number of Bedrooms, Number of Bathrooms,
House size, Description and Neighbourhood Description

NOTE: Number of Bedrooms and Bathrooms should be integers only
Make sure to be creative, talented

Example: 
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
"""
#promptTemplate2 = PromptTemplate(input_variables = ["num_listings"],template = template)
promptTemplate = PromptTemplate.from_template(template)


In [5]:
num_listings = 10

In [6]:
response = model(promptTemplate.format(num_listings = num_listings))

In [7]:
ListingSeperated = response.strip().split("\n\n")

for eachListing in ListingSeperated:
    # Replace \n with actual new lines
    formatted = eachListing.replace('\n','\n')
    print(formatted)  

    
    # Print a separator line after descriptions
    if "Neighborhood Description:" in formatted:
        print("\n" + "-"*50 + "\n")  # Separator for readability

1. Neighborhood: Sunset Heights
Price: $1,200,000
Bedrooms: 4
Bathrooms: 3
House Size: 2,500 sqft
Description: Step into luxury living in this stunning 4-bedroom, 3-bathroom home in Sunset Heights. The spacious open floor plan is perfect for entertaining, with a gourmet kitchen featuring high-end appliances and a large island. The master suite boasts a spa-like bathroom with a soaking tub and walk-in shower. Enjoy the beautiful sunset views from the backyard patio, complete with a built-in BBQ and fire pit.
Neighborhood Description: Sunset Heights is known for its upscale homes and breathtaking views of the city skyline. Residents enjoy easy access to trendy restaurants, boutique shops, and hiking trails in the nearby mountains.

--------------------------------------------------

2. Neighborhood: Lakeside Estates
Price: $950,000
Bedrooms: 5
Bathrooms: 4
House Size: 3,000 sqft
Description: This spacious 5-bedroom, 4-bathroom home in Lakeside Estates offers luxury living at its finest. 

In [8]:
text = response

# Split the text by the neighborhood number pattern
import re

neighborhoods = re.split(r'\n(?=\d+\.)', text)

# Create a list to store the formatted neighborhoods
formatted_neighborhoods = []

for i in range(num_listings):  # Adjust this number for more neighborhoods
    formatted_neighborhoods.append(neighborhoods[i].strip().replace('\n', ' '))
    #formatted_neighborhoods.append(neighborhoods[i].strip().split('\n'))


# Print the result
#print(formatted_neighborhoods)


In [9]:
formatted_neighborhoods[9]

'10. Neighborhood: Oakridge Meadows Price: $700,000 Bedrooms: 3 Bathrooms: 2 House Size: 1,800 sqft  Description: Welcome to this charming 3-bedroom, 2-bathroom home in Oakridge Meadows. The bright and airy living room features a fireplace and hardwood floors, while the updated kitchen includes granite countertops and stainless steel appliances. The backyard patio is perfect for outdoor dining and relaxing in the peaceful surroundings.  Neighborhood Description: Oakridge Meadows is a family-friendly community with tree-lined streets and a community park. Residents can enjoy local cafes, farmers markets, and easy access to public transportation for commuting to downtown.'

In [42]:
with open('listings.txt', 'w') as file:
    for listing in formatted_neighborhoods:
        file.write(listing)
        file.write("\n" + "-"*50 + "\n")

In [10]:
pip install sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install --upgrade sentence-transformers huggingface-hub

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
!pip install sentence-transformers
!pip install huggingface-hub

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [12]:
import chromadb
from chromadb.utils import embedding_functions
chroma_client = chromadb.Client()

In [14]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")
collection = chroma_client.create_collection(name="real_estate_listing", embedding_function= sentence_transformer_ef)

/home/student/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/home/student/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
ids = []
for i in range(1, num_listings+1):
    ids.append(str(i))

ids

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [16]:
collection.add(
    documents = formatted_neighborhoods
    ,ids = ids
)

In [17]:
collection.get(include = ['documents'])

{'ids': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'embeddings': None,
 'metadatas': None,
 'documents': ['1. Neighborhood: Sunset Heights Price: $1,200,000 Bedrooms: 4 Bathrooms: 3 House Size: 2,500 sqft  Description: Step into luxury living in this stunning 4-bedroom, 3-bathroom home in Sunset Heights. The spacious open floor plan is perfect for entertaining, with a gourmet kitchen featuring high-end appliances and a large island. The master suite boasts a spa-like bathroom with a soaking tub and walk-in shower. Enjoy the beautiful sunset views from the backyard patio, complete with a built-in BBQ and fire pit.  Neighborhood Description: Sunset Heights is known for its upscale homes and breathtaking views of the city skyline. Residents enjoy easy access to trendy restaurants, boutique shops, and hiking trails in the nearby mountains.',
  '2. Neighborhood: Lakeside Estates Price: $950,000 Bedrooms: 5 Bathrooms: 4 House Size: 3,000 sqft  Description: This spacious 5-bedroom,

In [35]:
questions = [   
"How big do you want your house to be?" ,
"What are 3 most important things for you in choosing this property?", 
"Which amenities would you like?", 
"Which transportation options are important to you?",
"How urban do you want your neighborhood to be?",   
            ]

BuyerPreferenceLIST = []
BuyerPreferenceString = ""
for ques in questions:
    answer = input(ques+ " ")
    BuyerPreferenceLIST.append(answer)
    BuyerPreferenceString = BuyerPreferenceString + answer
    


How big do you want your house to be? A comfortable three-bedroom house with a spacious kitchen and a cozy living room.
What are 3 most important things for you in choosing this property? A quiet neighborhood, good local schools, and convenient shopping options.
Which amenities would you like? A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.
Which transportation options are important to you? Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.
How urban do you want your neighborhood to be? A balance between suburban tranquility and access to urban amenities like restaurants and theaters.


In [36]:
BuyerPreferenceString

'A comfortable three-bedroom house with a spacious kitchen and a cozy living room.A quiet neighborhood, good local schools, and convenient shopping options.A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.A balance between suburban tranquility and access to urban amenities like restaurants and theaters.'

In [26]:
results = collection.query(
    query_texts=BuyerPreferenceString, # Chroma will embed this for you
    n_results=2, # how many results to return
    include = ['documents', 'distances']
)
print(results)

{'ids': [['3', '8']], 'distances': [[0.8516377210617065, 0.8964232206344604]], 'metadatas': None, 'embeddings': None, 'documents': [['3. Neighborhood: Oakwood Park Price: $700,000 Bedrooms: 3 Bathrooms: 2 House Size: 1,800 sqft  Description: Welcome home to this charming 3-bedroom, 2-bathroom house in Oakwood Park. The cozy living room features a fireplace and hardwood floors, while the updated kitchen includes granite countertops and stainless steel appliances. The backyard patio is perfect for outdoor dining and entertaining.  Neighborhood Description: Oakwood Park is a family-friendly neighborhood with tree-lined streets and a community park. Residents can enjoy local cafes, farmers markets, and easy access to public transportation for commuting to downtown.', '8. Neighborhood: Maple Grove Estates Price: $900,000 Bedrooms: 5 Bathrooms: 4 House Size: 2,800 sqft  Description: This spacious 5-bedroom, 4-bathroom home in Maple Grove Estates offers luxury living in a peaceful setting. Th

In [30]:
results['documents'][0][0]

'3. Neighborhood: Oakwood Park Price: $700,000 Bedrooms: 3 Bathrooms: 2 House Size: 1,800 sqft  Description: Welcome home to this charming 3-bedroom, 2-bathroom house in Oakwood Park. The cozy living room features a fireplace and hardwood floors, while the updated kitchen includes granite countertops and stainless steel appliances. The backyard patio is perfect for outdoor dining and entertaining.  Neighborhood Description: Oakwood Park is a family-friendly neighborhood with tree-lined streets and a community park. Residents can enjoy local cafes, farmers markets, and easy access to public transportation for commuting to downtown.'

In [31]:
results['documents'][0][1]

'8. Neighborhood: Maple Grove Estates Price: $900,000 Bedrooms: 5 Bathrooms: 4 House Size: 2,800 sqft  Description: This spacious 5-bedroom, 4-bathroom home in Maple Grove Estates offers luxury living in a peaceful setting. The gourmet kitchen features custom cabinets and high-end appliances, while the master suite includes a sitting area and en-suite bathroom with a soaking tub. The backyard patio is perfect for outdoor dining and relaxing in the hot tub.  Neighborhood Description: Maple Grove Estates is a prestigious community with tree-lined streets and well-maintained homes. Residents can enjoy local parks, golf courses, and shopping centers, as well as top-rated schools and easy access to major highways for commuting.'

In [33]:
augmenting = """
Your role is a real estate expert
I will provide you with a buyer preferences and 
a real estate listing description,
make sure to change the real estate description 
to match the buyer preferences.

NOTE: You have to Maintain Factual Integrity, 
don't change number of bedrooms or bathrooms etc.

Enhance this description : {real_estate_description}
to match Buyer Preference: {Buyer_Preference}

"""
AugmentingPrompt = PromptTemplate.from_template(augmenting)

In [37]:
response2 = model(AugmentingPrompt.format(real_estate_description = results['documents'][0][1], Buyer_Preference = BuyerPreferenceString))

In [38]:
response2

'Enhanced Description: This cozy 5-bedroom, 4-bathroom home in Maple Grove Estates offers a comfortable living space with a spacious kitchen and a cozy living room. The gourmet kitchen features custom cabinets and high-end appliances, while the master suite includes a sitting area and en-suite bathroom with a soaking tub. The backyard patio is perfect for outdoor dining and relaxing in the hot tub, with space for gardening and a two-car garage. The modern, energy-efficient heating system ensures year-round comfort. \n\nMaple Grove Estates is a quiet neighborhood with tree-lined streets and well-maintained homes, offering good local schools and convenient shopping options. Residents can enjoy local parks, golf courses, and easy access to major highways for commuting. The area also provides easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads, striking a perfect balance between suburban tranquility and access to urban amenities like restaurants and th